## Day 8

### Part 1
Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
For example, consider the following program:

- nop +0
- acc +1
- jmp +4
- acc +3
- jmp -3
- acc -99
- acc +1
- jmp -4
- acc +6

These instructions are visited in this order:

- nop +0  | 1
- acc +1  | 2, 8(!)
- jmp +4  | 3
- acc +3  | 6
- jmp -3  | 7
- acc -99 |
- acc +1  | 4
- jmp -4  | 5
- acc +6  |

First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [1]:
# import packages
import pandas as pd
import numpy as np

In [2]:
# read in data
instructions = pd.read_csv('input_data/Day8.txt', sep=" ", header=None)

In [3]:
instructions.shape

(633, 2)

In [4]:
instructions.columns=['instruction','steps']

In [5]:
instructions['has_run']=False

In [6]:
instructions.head()

,instruction,steps,has_run
0,acc,37,False
1,acc,-1,False
2,nop,512,False
3,acc,0,False
4,jmp,60,False


In [7]:
def process_instruction(instruction, steps):
    '''takes in an instruction and value or steps and modifies global variables
    acc_count and position, returning a flag indicating if new position has been
    run before'''
    global count, pos
    
    # Set to True has_run flag for this position
    instructions.at[pos,'has_run'] = True
    
    if instruction=='acc':
        # increase acc_count by steps and position by 1
        count += steps
        pos += 1
    elif instruction == 'jmp':
        # do not touch acc_count but jump forward or backward number of steps
        pos += steps
    else:
        # do nothing but go forward one position
        pos += 1
    
    return True if instructions.iloc[pos]['has_run'] else False

In [8]:
# these are global variables
pos = 0
count = 0
instructions['has_run']=False

flag = False
while not flag:
    flag = process_instruction(instructions.iloc[pos]['instruction'], instructions.iloc[pos]['steps'])
    
print('count: ', count)

count:  1684


### Part 2
After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

- nop +0
- acc +1
- jmp +4
- acc +3
- jmp -3
- acc -99
- acc +1
- jmp -4
- acc +6

If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

- nop +0  | 1
- acc +1  | 2
- jmp +4  | 3
- acc +3  |
- jmp -3  |
- acc -99 |
- acc +1  | 4
- nop -4  | 5
- acc +6  | 6

After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?


In [9]:
def process_instruction(instruction, steps):
    '''takes in an instruction and value or steps and modifies global variables
    acc_count and position, returning a flag indicating if new position has been
    run before'''
    global count, pos
    
    # Set to True has_run flag for this position
    test_instructions.at[pos,'has_run'] = True
    
#    print(instruction, pos, steps, count)
    if instruction=='acc':
        # increase acc_count by steps and position by 1
        count += steps
        pos += 1
    elif instruction == 'jmp':
        # do not touch acc_count but jump forward or backward number of steps
        pos += steps
    else:
        # do nothing but go forward one position
        pos += 1
    
    # We want to return stop processing the instructions under two circumstances, 
    # if we've hit the bottom of the instructions or if the loop is infinite
    if (pos == last_pos + 1):
        return True
    elif test_instructions.iloc[pos]['has_run']:
        return True
    else:
        return False

In [10]:
# Put this inside a loop where we loop through our dataframe, changing one nop or jmp instruction at a time

# Start at the first instruction
test_pos = 0
test_flag = True
last_pos = instructions.shape[0]-1

while test_flag:
    
    # make a copy of our dataframe with the original set of instructions
    test_instructions = instructions.copy()
    # grab the next instruction to test
    test_inst = test_instructions.iloc[test_pos]['instruction']
    
    if (test_inst == 'jmp' or test_inst == 'nop'):
        
        # swap instructions
        if test_inst == 'jmp':
            test_instructions.at[test_pos,'instruction'] = 'nop' 
        else:
            test_instructions.at[test_pos,'instruction'] = 'jmp'
 
        # now test this set of instructions
        pos = 0
        count = 0
        test_instructions['has_run']=False
        flag = False
        while not flag:
            flag = process_instruction(test_instructions.iloc[pos]['instruction'], test_instructions.iloc[pos]['steps'])
 
        # we're back from testing that set of instructions
        # Let's see if it finished due to an infinite loop or whether it actually completed
        if pos == last_pos+1:
            print("We did it!  Count = ", count)
            test_flag = False
                
    test_pos += 1
    if test_pos == last_pos:
        # We've reached the bottom of the instructions
        test_flag = False


We did it!  Count =  2188
